In [1]:
import abc_py as abcPy
import numpy as np
import torch

####################################################################################################################

circuit_prefix = '/home/xcf/Desktop/ML_project/InitialAIG/train/'
aig_prefix = '/home/xcf/Desktop/ML_project/task1/aig_files/'

libFile = '/home/xcf/Desktop/ML_project/lib/7nm/7nm.lib'
logFile = 'yosys_process.log'

synthesisOpToPosDic = {
    0: 'refactor',
    1: 'refactor -z',
    2: 'rewrite',
    3: 'rewrite -z',
    4: 'resub',
    5: 'resub -z',
    6: 'balance'
}

# Get aig function. Call yosys-abc to solve the aig.
def get_aig(state):
    circuitName, actions = state.split('_')
    circuitPath = circuit_prefix + circuitName + '.aig'
    nextState = aig_prefix + state + '.aig'  # current AIG file
    nextBench = aig_prefix + state + '.bench'  # current bench file

    actionCmd = ''
    for action in actions:
        actionCmd += (synthesisOpToPosDic[int(action)] + ';')
    abcRunCmd = "/home/xcf/Desktop/oss-cad-suite/bin/yosys-abc -c \"read "+circuitPath + ";" + actionCmd + \
        " read_lib " + libFile + "; write " + \
        nextState +  "; print_stats \"> " + logFile 
    
    # print(abcRunCmd)
    os.system(abcRunCmd)

    return nextState

####################################################################################################################

_abc = abcPy.AbcInterface()
_abc.start()

# Get data function. Use abc_py to get the tensor form data from .aig file.
def get_data(state):
    # Current aig in state.aig
    _abc.read(state)
    data = {}
    
    numNodes = _abc.numNodes()
    data['node_type'] = np.zeros(numNodes, dtype = int)
    data['num_inverted_predecessors'] = np.zeros(numNodes, dtype = int)
    
    edge_src_index = []
    edge_target_index = []
    
    for nodeIdx in range(numNodes):
        aigNode = _abc.aigNode(nodeIdx)
        nodeType = aigNode.nodeType()
        data['num_inverted_predecessors'][nodeIdx] = 0
        if nodeType == 0 or nodeType == 2 :
            data['node_type'][nodeIdx] = 0
        elif nodeType == 1 :
            data['node_type'][nodeIdx] = 1
        else :
            data['node_type'][nodeIdx] = 2
            if nodeType == 4 :
                data['num_inverted_predecessors'][nodeIdx] = 1
            if nodeType == 5 :
                data['num_inverted_predecessors'][nodeIdx] = 2
        if (aigNode.hasFanin0()) :
            fanin = aigNode.fanin0()
            edge_src_index.append(nodeIdx)
            edge_target_index.append(fanin)
        if (aigNode.hasFanin1()) :
            fanin = aigNode.fanin1()
            edge_src_index.append(nodeIdx)
            edge_target_index.append(fanin)
            
    data['edge_index'] = torch.tensor([edge_src_index, edge_target_index], dtype = torch.long)
    data['node_type'] = torch.tensor(data['node_type'])
    data['num_inverted_predecessors'] = torch.tensor(data['num_inverted_predecessors'])
    data['nodes'] = numNodes

    return data
    


In [2]:
####  The function to parse aig  file with name and actions  ####

import pickle
import os
import torch
import torch_geometric
from torch_geometric.data import Data as GEOData

dir = "/home/xcf/Desktop/ML_project/task1/project_data"

def parse_aig(path):
    state = ''
    target = 0
    with open(path, 'rb') as f:
        pkl_file = pickle.load(f)
        for i, (name, vec) in enumerate(pkl_file.items()):
            if name == 'input':
                state = vec[-1]
            elif name == 'target':
                target = vec[-1]

        nextState = get_aig(state)
        rawData = get_data(nextState)
        
        # Generate torch.geometric.data.Data type
        y_label = torch.tensor(target)
        x_features = torch.stack((rawData['node_type'], rawData['num_inverted_predecessors']), dim = 0)
        edge_index = rawData['edge_index']

        this_data = GEOData(x = x_features.t().float(), edge_index = edge_index, y = y_label)
    return this_data

/home/xcf/anaconda3/lib/python3.11/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/xcf/anaconda3/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/xcf/anaconda3/lib/python3.11/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/xcf/anaconda3/lib/python3.11/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [6]:
data_test = parse_aig("/home/xcf/Desktop/ML_project/task1/project_data/adder_1.pkl")
print(data_test)

Data(x=[1279, 2], edge_index=[2, 1915], y=0.1595180630683899)
Data(x=[3, 1], edge_index=[2, 2], y=[3])


In [18]:
####  Task 1 : GCN for graph evaluation  ####

import torch
import random
random.seed(42)

dir = "/home/xcf/Desktop/ML_project/task1/project_data"
data_dir = "/home/xcf/Desktop/ML_project/task1/graph_data_files/"
path_pool = []

for filename in os.listdir(dir):
    this_path = os.path.join(dir, filename)
    path_pool.append(this_path)

print("done")

random.shuffle(path_pool)

sz = len(path_pool)
train_data = []
test_data = []

ind = 0

for i in range(10000):
    this_data = parse_aig(path_pool[i])

    flag = False
    sz = this_data.x.shape[0]
    for ts in this_data.edge_index:
        for element in ts:
            if element >= sz:
                flag = True
                break
        if flag:
            break
    if flag:
        continue
        
    data_path = data_dir + "data_" + str(ind) + ".pt"
    torch.save(this_data, data_path)    
    ind = ind + 1

    if ((i+1) % 100 == 0):
        print("Operating data #", (i+1))
print(ind)

done
Operating data # 100
Operating data # 200
Operating data # 300
Operating data # 400
Operating data # 500
Operating data # 700
Operating data # 900
Operating data # 1000
Operating data # 1200
Operating data # 1300
Operating data # 1400
Operating data # 1500
Operating data # 1700
Operating data # 1900
Operating data # 2100
Operating data # 2200
Operating data # 2300
Operating data # 2500
Operating data # 2600
Operating data # 2700
Operating data # 2900
Operating data # 3400
Operating data # 3500
Operating data # 3600
Operating data # 3700
Operating data # 3900
Operating data # 4000
Operating data # 4100
Operating data # 4200
Operating data # 4300
Operating data # 4400
Operating data # 4500
Operating data # 4600
Operating data # 4700
Operating data # 4800
Operating data # 4900
Operating data # 5000
Operating data # 5100
Operating data # 5400
Operating data # 5500
Operating data # 5600
Operating data # 5700
Operating data # 5800
Operating data # 5900
Operating data # 6100
Operating da

KeyboardInterrupt: 